# 1. Data Exploration

The first section we will look at techniques to load and explore the data to be used for machine learning. The purpose of this exploration to gain an understanding of the data and problem to make sensible choices in building the machine learning pipeline.

NOOA template sections
## Overview 

### Prerequisites
xx

### Learning Outcomes 
xx

## Tutorial 
xx

## Excercise 
xx

## Examples of use
xx
 
## Next steps
xx

## Dataset Info
xx

## References
xx


## Load and explore tabular dataset - falkland islands rotors

## Load and explore gridded obs dataset - UK radar

## Load and explore gridded model data - ERA5